# Import Library

In [ ]:
import cv2 as cv
import numpy as np
import pandas as pd
import os
import random
import shutil
import xml.etree.ElementTree as ET
import torch

from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
from IPython.display import Image, clear_output

# Dataset

## Download Dataset

In [ ]:
!pip install kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import json
token = {"username":"ssusie2737","key":"e9b8d049190b8efa5f5ba6b02d2a9a66"}
with open('/content/kaggle.json', 'w') as file:
  json.dump(token, file)

In [ ]:
!mkdir -p ~/.kaggle

In [ ]:
!cp /content/kaggle.json ~/.kaggle/

In [ ]:
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle config set -n path -v /content

- path is now set to: /content


In [ ]:
!kaggle datasets download -d jessicali9530/stanford-dogs-dataset

100% 748M/750M [00:06<00:00, 112MB/s]
100% 750M/750M [00:06<00:00, 126MB/s]


In [ ]:
!unzip /content/datasets/jessicali9530/stanford-dogs-dataset/stanford-dogs-dataset.zip

Streaming output truncated to the last 5000 lines.
  inflating: images/Images/n02108089-boxer/n02108089_11687.jpg  
  inflating: images/Images/n02108089-boxer/n02108089_117.jpg  
  inflating: images/Images/n02108089-boxer/n02108089_11807.jpg  
  inflating: images/Images/n02108089-boxer/n02108089_11875.jpg  
  inflating: images/Images/n02108089-boxer/n02108089_122.jpg  
  inflating: images/Images/n02108089-boxer/n02108089_12232.jpg  
  inflating: images/Images/n02108089-boxer/n02108089_125.jpg  
  inflating: images/Images/n02108089-boxer/n02108089_12738.jpg  
  inflating: images/Images/n02108089-boxer/n02108089_12739.jpg  
  inflating: images/Images/n02108089-boxer/n02108089_12827.jpg  
  inflating: images/Images/n02108089-boxer/n02108089_13340.jpg  
  inflating: images/Images/n02108089-boxer/n02108089_13526.jpg  
  inflating: images/Images/n02108089-boxer/n02108089_1353.jpg  
  inflating: images/Images/n02108089-boxer/n02108089_1355.jpg  
  inflating: images/Images/n02108089-boxer/n021

## Reorg Directory Structure

In [ ]:
os.mkdir('/content/dataset')
os.mkdir('/content/dataset/images')
os.mkdir('/content/dataset/annotations')

In [ ]:
sources = []
for root, dirs, files in os.walk('/content/images/Images'):
  for dir in dirs:
    subdir = os.path.join(root, dir)
    for subroot, subdirs, subfiles in os.walk(subdir):
      for subfile in subfiles:
        sources.append(os.path.join(subroot, subfile))

target = '/content/dataset/images'

for source in sources:
  shutil.copy(source, target)

In [ ]:
sources = []
for root, dirs, files in os.walk('/content/annotations/Annotation'):
  for dir in dirs:
    subdir = os.path.join(root, dir)
    for subroot, subdirs, subfiles in os.walk(subdir):
      for subfile in subfiles:
        sources.append(os.path.join(subroot, subfile))

target = '/content/dataset/annotations'

for source in sources:
  shutil.copy(source, target)

In [ ]:
os.mkdir('/content/dataset/labels')

## Convert Annotations

In [ ]:
# Create a set of classes
# Expected to 120 dog breeds
classes = set()
paths = []
for root, dirs, files in os.walk('/content/dataset/annotations'):
  for file in files:
    paths.append(os.path.join(root, file))

for path in paths:
    tree = ET.parse(path)
    root = tree.getroot()
    for obj in root.iter('object'):
        cls = obj.find('name').text
        classes.add(cls)

In [ ]:
len(classes)

120

In [ ]:
classes

{'Afghan_hound',
 'African_hunting_dog',
 'Airedale',
 'American_Staffordshire_terrier',
 'Appenzeller',
 'Australian_terrier',
 'Bedlington_terrier',
 'Bernese_mountain_dog',
 'Blenheim_spaniel',
 'Border_collie',
 'Border_terrier',
 'Boston_bull',
 'Bouvier_des_Flandres',
 'Brabancon_griffon',
 'Brittany_spaniel',
 'Cardigan',
 'Chesapeake_Bay_retriever',
 'Chihuahua',
 'Dandie_Dinmont',
 'Doberman',
 'English_foxhound',
 'English_setter',
 'English_springer',
 'EntleBucher',
 'Eskimo_dog',
 'French_bulldog',
 'German_shepherd',
 'German_short-haired_pointer',
 'Gordon_setter',
 'Great_Dane',
 'Great_Pyrenees',
 'Greater_Swiss_Mountain_dog',
 'Ibizan_hound',
 'Irish_setter',
 'Irish_terrier',
 'Irish_water_spaniel',
 'Irish_wolfhound',
 'Italian_greyhound',
 'Japanese_spaniel',
 'Kerry_blue_terrier',
 'Labrador_retriever',
 'Lakeland_terrier',
 'Leonberg',
 'Lhasa',
 'Maltese_dog',
 'Mexican_hairless',
 'Newfoundland',
 'Norfolk_terrier',
 'Norwegian_elkhound',
 'Norwich_terrier',
 '

In [ ]:
class_list = []
for cls in classes:
  class_list.append(cls)

In [ ]:
def normalize(size, box):
    dw = 1./size[0]    
    dh = 1./size[1]     
    x = (box[0] + box[1])/2.0   
    y = (box[2] + box[3])/2.0   
    w = box[1] - box[0]         
    h = box[3] - box[2]         
    x = x*dw    
    w = w*dw    
    y = y*dh    
    h = h*dh    
    return (x, y, w, h)    

In [ ]:
def convert_annotation(file_name):
    in_file = open('/content/dataset/annotations/%s' % (file_name), encoding='utf-8')
    out_file = open('/content/dataset/labels/%s.txt' % (file_name), 'w', encoding='utf-8')
    tree = ET.parse(in_file)
    root = tree.getroot()
    size = root.find('size')
    w = int(size.find('width').text)
    h = int(size.find('height').text)
    for obj in root.iter('object'):
        cls = obj.find('name').text
        if cls not in class_list:
            continue
        cls_id = class_list.index(cls)
        xmlbox = obj.find('bndbox')
        b = (float(xmlbox.find('xmin').text), 
             float(xmlbox.find('xmax').text), 
             float(xmlbox.find('ymin').text),
             float(xmlbox.find('ymax').text))
        print(file_name, cls, b)
        bb = normalize((w, h), b)
        out_file.write(str(cls_id) + " " + " ".join([str(a) for a in bb]) + '\n')

In [ ]:
for root, dirs, files in os.walk('/content/dataset/annotations'):
  for file in files:
    convert_annotation(file)

Streaming output truncated to the last 5000 lines.
n02086079_10294 Pekinese (36.0, 359.0, 24.0, 230.0)
n02093754_1557 Border_terrier (97.0, 384.0, 91.0, 322.0)
n02093991_3061 Irish_terrier (190.0, 323.0, 34.0, 276.0)
n02100877_1062 Irish_setter (52.0, 438.0, 29.0, 298.0)
n02088094_93 Afghan_hound (281.0, 413.0, 224.0, 456.0)
n02091635_4068 otterhound (22.0, 137.0, 3.0, 114.0)
n02093754_186 Border_terrier (0.0, 370.0, 0.0, 498.0)
n02088364_16508 beagle (238.0, 421.0, 52.0, 323.0)
n02094433_7445 Yorkshire_terrier (0.0, 458.0, 22.0, 374.0)
n02115641_9977 dingo (71.0, 234.0, 48.0, 185.0)
n02090622_7292 borzoi (274.0, 361.0, 148.0, 344.0)
n02099849_3753 Chesapeake_Bay_retriever (140.0, 370.0, 74.0, 226.0)
n02091032_11660 Italian_greyhound (204.0, 423.0, 159.0, 345.0)
n02091032_11660 Italian_greyhound (104.0, 292.0, 135.0, 345.0)
n02102973_5052 Irish_water_spaniel (1.0, 188.0, 121.0, 299.0)
n02102973_5052 Irish_water_spaniel (98.0, 279.0, 8.0, 288.0)
n02105162_6459 malinois (17.0, 456.0, 240

## Divide Train/Val/Test Sets

In [ ]:
!rm -rf dataset_in_use

In [ ]:
os.mkdir('/content/dataset_in_use')

os.mkdir('/content/dataset_in_use/train')
os.mkdir('/content/dataset_in_use/train/images')
os.mkdir('/content/dataset_in_use/train/labels')

os.mkdir('/content/dataset_in_use/val')
os.mkdir('/content/dataset_in_use/val/images')
os.mkdir('/content/dataset_in_use/val/labels')

os.mkdir('/content/dataset_in_use/test')
os.mkdir('/content/dataset_in_use/test/images')
os.mkdir('/content/dataset_in_use/test/labels')

In [ ]:
def populate_datase_in_use(num_train, num_val, num_test):

  train_list = []
  val_list = []
  test_list = []

  train_image_path = '/content/dataset_in_use/train/images'
  train_label_path = '/content/dataset_in_use/train/labels'
  val_image_path = '/content/dataset_in_use/val/images'
  val_label_path = '/content/dataset_in_use/val/labels'
  test_image_path = '/content/dataset_in_use/test/images'
  test_label_path = '/content/dataset_in_use/test/labels'

  dirs = os.listdir('/content/images/Images')

  # populate train set
  for dir in dirs:
    count_train = 0
    subdir = '/content/images/Images/' + dir
    for file in os.listdir(subdir):
      train_list.append(file)
      shutil.copy(subdir + '/' + file, train_image_path)
      os.remove(subdir + '/' + file)
      count_train += 1
      if(count_train == num_train):
        break
  print(len(train_list))
  for file in train_list:
    shutil.copy('/content/dataset/labels/' + file.split('.')[0] + '.txt', train_label_path)

  # populate val set
  for dir in dirs:
    count_val = 0
    subdir = '/content/images/Images/' + dir
    for file in os.listdir(subdir):
      val_list.append(file)
      shutil.copy(subdir + '/' + file, val_image_path)
      os.remove(subdir + '/' + file)
      count_val += 1
      if(count_val == num_val):
        break
  print(len(val_list))
  for file in val_list:
    shutil.copy('/content/dataset/labels/' + file.split('.')[0] + '.txt', val_label_path)

  # populate test set
  for dir in dirs:
    count_test = 0
    subdir = '/content/images/Images/' + dir
    for file in os.listdir(subdir):
      test_list.append(file)
      shutil.copy(subdir + '/' + file, test_image_path)
      os.remove(subdir + '/' + file)
      count_test += 1
      if(count_test == num_test):
        break
  print(len(test_list))
  for file in test_list:
    shutil.copy('/content/dataset/labels/' + file.split('.')[0] + '.txt', test_label_path)

In [ ]:
populate_datase_in_use(10, 2, 2) # change the number for desired train/val/test size

1200
240
240


# YOLOv5

## Install

In [ ]:
!git clone https://github.com/ultralytics/yolov5.git
!pip install -r yolov5/requirements.txt

Cloning into 'yolov5'...
remote: Enumerating objects: 15356, done.
remote: Counting objects: 100% (1/1), done.
remote: Total 15356 (delta 0), reused 1 (delta 0), pack-reused 15355
Receiving objects: 100% (15356/15356), 14.33 MiB | 20.70 MiB/s, done.
Resolving deltas: 100% (10503/10503), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 KB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 KB 8.3 MB/s eta 0:00:00


## Train

In [ ]:
class_list

['malinois',
 'vizsla',
 'Rhodesian_ridgeback',
 'Bernese_mountain_dog',
 'Mexican_hairless',
 'dingo',
 'Rottweiler',
 'Leonberg',
 'Irish_wolfhound',
 'Norwich_terrier',
 'borzoi',
 'African_hunting_dog',
 'Siberian_husky',
 'otterhound',
 'toy_poodle',
 'German_shepherd',
 'Labrador_retriever',
 'Samoyed',
 'Scottish_deerhound',
 'whippet',
 'affenpinscher',
 'Kerry_blue_terrier',
 'English_setter',
 'boxer',
 'keeshond',
 'curly-coated_retriever',
 'wire-haired_fox_terrier',
 'beagle',
 'bloodhound',
 'Yorkshire_terrier',
 'Brabancon_griffon',
 'Norfolk_terrier',
 'basenji',
 'Irish_water_spaniel',
 'Cardigan',
 'giant_schnauzer',
 'toy_terrier',
 'American_Staffordshire_terrier',
 'Sussex_spaniel',
 'miniature_schnauzer',
 'miniature_poodle',
 'French_bulldog',
 'Chihuahua',
 'pug',
 'Italian_greyhound',
 'Walker_hound',
 'English_springer',
 'kelpie',
 'black-and-tan_coonhound',
 'Australian_terrier',
 'English_foxhound',
 'Tibetan_mastiff',
 'schipperke',
 'German_short-haired_p

In [ ]:
class_str = ''

class_str += '['

for cls in class_list:
  class_str += '\''
  class_str += cls
  class_str += '\''
  class_str += ','

class_str = class_str[:-1]

class_str += ']'

class_str

"['malinois','vizsla','Rhodesian_ridgeback','Bernese_mountain_dog','Mexican_hairless','dingo','Rottweiler','Leonberg','Irish_wolfhound','Norwich_terrier','borzoi','African_hunting_dog','Siberian_husky','otterhound','toy_poodle','German_shepherd','Labrador_retriever','Samoyed','Scottish_deerhound','whippet','affenpinscher','Kerry_blue_terrier','English_setter','boxer','keeshond','curly-coated_retriever','wire-haired_fox_terrier','beagle','bloodhound','Yorkshire_terrier','Brabancon_griffon','Norfolk_terrier','basenji','Irish_water_spaniel','Cardigan','giant_schnauzer','toy_terrier','American_Staffordshire_terrier','Sussex_spaniel','miniature_schnauzer','miniature_poodle','French_bulldog','Chihuahua','pug','Italian_greyhound','Walker_hound','English_springer','kelpie','black-and-tan_coonhound','Australian_terrier','English_foxhound','Tibetan_mastiff','schipperke','German_short-haired_pointer','Staffordshire_bullterrier','Scotch_terrier','Border_collie','redbone','Brittany_spaniel','standa

In [ ]:
config = """train: /content/dataset_in_use/train/images
val: /content/dataset_in_use/val/images
test: /content/dataset_in_use/test/images

nc: 120
names: ['Mexican_hairless','Border_collie','Brabancon_griffon','Appenzeller','chow','papillon','briard','Staffordshire_bullterrier','Gordon_setter','Airedale','kelpie','EntleBucher','Scottish_deerhound','borzoi','Kerry_blue_terrier','Italian_greyhound','Saluki','Bedlington_terrier','Welsh_springer_spaniel','malamute','curly-coated_retriever','komondor','miniature_schnauzer','Pembroke','Afghan_hound','flat-coated_retriever','Irish_wolfhound','schipperke','black-and-tan_coonhound','toy_terrier','malinois','affenpinscher','Blenheim_spaniel','bluetick','Norwich_terrier','clumber','West_Highland_white_terrier','Shetland_sheepdog','kuvasz','English_springer','Pomeranian','Scotch_terrier','toy_poodle','Weimaraner','Great_Pyrenees','Chihuahua','miniature_poodle','Great_Dane','Tibetan_terrier','Bouvier_des_Flandres','Leonberg','otterhound','basset','pug','Labrador_retriever','collie','Lakeland_terrier','Walker_hound','Rhodesian_ridgeback','basenji','dhole','German_short-haired_pointer','Sussex_spaniel','Irish_terrier','Maltese_dog','Bernese_mountain_dog','soft-coated_wheaten_terrier','Norfolk_terrier','vizsla','standard_schnauzer','groenendael','American_Staffordshire_terrier','Ibizan_hound','Border_terrier','Old_English_sheepdog','boxer','Shih-Tzu','cairn','Cardigan','English_setter','Australian_terrier','giant_schnauzer','Eskimo_dog','miniature_pinscher','African_hunting_dog','Norwegian_elkhound','Irish_water_spaniel','German_shepherd','French_bulldog','dingo','golden_retriever','Brittany_spaniel','English_foxhound','Chesapeake_Bay_retriever','Doberman','bloodhound','Greater_Swiss_Mountain_dog','beagle','whippet','Irish_setter','Saint_Bernard','Newfoundland','keeshond','Siberian_husky','Samoyed','bull_mastiff','Dandie_Dinmont','redbone','Yorkshire_terrier','Tibetan_mastiff','Japanese_spaniel','silky_terrier','cocker_spaniel','standard_poodle','Sealyham_terrier','Lhasa','Rottweiler','wire-haired_fox_terrier','Boston_bull','Pekinese'] """

In [ ]:
yaml = open('/content/yolov5/data/dog.yaml', 'w')
yaml.write(config)
yaml.close()

In [ ]:
!python yolov5/train.py --img 160 --batch 32 --epochs 50 --data yolov5/data/dog.yaml --weights yolov5s.pt --cache

train: weights=yolov5s.pt, cfg=, data=yolov5/data/dog.yaml, hyp=yolov5/data/hyps/hyp.scratch-low.yaml, epochs=50, batch_size=32, imgsz=160, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=yolov5/runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v7.0-132-ga82132c Python-3.9.16 torch-1.13.1+cu116 CPU

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, de

## Test

In [ ]:
!python yolov5/detect.py --source /content/dataset_in_use/test/images --img 160 --conf 0.25 --data yolov5/data/dog.yaml --weight yolov5/runs/train/exp4/weights/best.pt

detect: weights=['yolov5/runs/train/exp4/weights/best.pt'], source=/content/dataset_in_use/test/images, data=yolov5/data/dog.yaml, imgsz=[160, 160], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=yolov5/runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-132-ga82132c Python-3.9.16 torch-1.13.1+cu116 CPU

Fusing layers... 
Model summary: 157 layers, 7333765 parameters, 0 gradients, 16.8 GFLOPs
image 1/240 /content/dataset_in_use/test/images/n02085620_10621.jpg: 96x160 (no detections), 44.1ms
image 2/240 /content/dataset_in_use/test/images/n02085620_382.jpg: 128x160 (no detections), 52.9ms
image 3/240 /content/dataset_in_use/test/images/n02085782_186.jpg: 160x128 (no detections), 51.8ms
image 4/240 /content/dataset_in_use/te

In [ ]:
!python yolov5/val.py --img 160 --conf 0.25 --data yolov5/data/dog.yaml --weight yolov5/runs/train/exp4/weights/best.pt

val: data=yolov5/data/dog.yaml, weights=['yolov5/runs/train/exp2/weights/best.pt'], batch_size=32, imgsz=160, conf_thres=0.25, iou_thres=0.6, max_det=300, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=yolov5/runs/val, name=exp, exist_ok=False, half=False, dnn=False
WARNING ⚠️ confidence threshold 0.25 > 0.001 produces invalid results
YOLOv5 🚀 v7.0-132-ga82132c Python-3.9.16 torch-1.13.1+cu116 CPU

Fusing layers... 
Model summary: 157 layers, 7333765 parameters, 0 gradients, 16.8 GFLOPs
val: Scanning /content/dataset_in_use/val/labels.cache... 240 images, 0 backgrounds, 0 corrupt: 100% 240/240 [00:00<?, ?it/s]
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100% 8/8 [00:11<00:00,  1.47s/it]
                   all        240        256          0          0          0          0
Speed: 0.4ms pre-process, 46.7ms inference, 0.3ms NMS per ima

## Export

In [ ]:
!python yolov5/export.py --weights yolov5/runs/train/exp4/weights/best.pt --include tflite

export: data=yolov5/data/coco128.yaml, weights=['yolov5/runs/train/exp4/weights/best.pt'], imgsz=[640, 640], batch_size=1, device=cpu, half=False, inplace=False, keras=False, optimize=False, int8=False, dynamic=False, simplify=False, opset=17, verbose=False, workspace=4, nms=False, agnostic_nms=False, topk_per_class=100, topk_all=100, iou_thres=0.45, conf_thres=0.25, include=['tflite']
YOLOv5 🚀 v7.0-132-ga82132c Python-3.9.16 torch-1.13.1+cu116 CPU

Fusing layers... 
Model summary: 157 layers, 7333765 parameters, 0 gradients, 16.8 GFLOPs

PyTorch: starting from yolov5/runs/train/exp4/weights/best.pt with output shape (1, 25200, 125) (14.2 MB)
2023-03-31 21:15:57.308775: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-31 21:15:59.382061: W tensorf

# Reference

[1] https://blog.csdn.net/qq_20880939/article/details/105613800

[2] https://blog.csdn.net/ai_faker/article/details/107692035?utm_medium=distribute.pc_relevant.none-task-blog-2~default~baidujs_title~default-0.topblog&spm=1001.2101.3001.4242.1&utm_relevant_index=3